In [1]:
# Importuri
import pandas as pd
import os


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/marmy/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/marmy/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/marmy/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/home/marmy/an

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/marmy/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/marmy/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/marmy/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/home/marmy/an

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [20]:
path = "./data/FoodData_Central_survey_food_csv_2024-10-31/"

csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]

csv_files.sort()

dataframes = [pd.read_csv(os.path.join(path, key)) for key in csv_files]

# Step 0 - Cleaning individual datasets

In [97]:
food_nutrient_ds = pd.read_csv(os.path.join(path, 'food_nutrient.csv'))

# Drop the columns from the original dataset that are full of null values
food_nutrient_ds = food_nutrient_ds.drop(columns=['data_points', 'derivation_id', 'min', 'max', 'median', 'footnote', 'min_year_acquired'])

food_nutrient_ds.head()

,id,fdc_id,nutrient_id,amount
0,34136169,2705384,301,125.000
1,34136178,2705384,319,57.000
2,34136185,2705384,337,0.000
3,34136218,2705384,628,0.002
4,34136219,2705384,629,0.000


In [ ]:
nutrient_ds = pd.read_csv(os.path.join(path, 'nutrient.csv'))

# Will be merged with food_nutrient_ds on nutrient_nbr => Nutrient number has to have the same type as nutrient_id from food_nutrient_ds
# Convert nutrient_nbr to int, filling NaN with -1
nutrient_ds['nutrient_nbr'] = nutrient_ds['nutrient_nbr'].fillna(-1).astype(int)

# Rank does not appear in the documentation of the database: https://fdc.nal.usda.gov/docs/Download_Field_Descriptions_Oct2020.pdf\
# So we will drop it for now to reduce computation and increase readability, speed and memory usage
nutrient_ds = nutrient_ds.drop(columns=['rank'])

# 'Id' won't be used for merging so we can drop it as well
nutrient_ds = nutrient_ds.drop(columns=['id'])

# Rename name to Nutrient_Name
# And: unit to Nutrient_Unit for better readability
nutrient_ds = nutrient_ds.rename(columns={'name': 'Nutrient_Name', 'unit_name': 'Nutrient_Unit'})

nutrient_ds.head()

,Nutrient_Name,Nutrient_Unit,nutrient_nbr
0,Energy (Atwater General Factors),KCAL,957
1,Energy (Atwater Specific Factors),KCAL,958
2,Solids,G,201
3,Nitrogen,G,202
4,Protein,G,203


In [101]:
food_ds = pd.read_csv(os.path.join(path, 'food.csv'))

# We will be using food to get food nutrients for each food item
# We only need the columns fdc_id (For the join) and description

food_ds = food_ds[['fdc_id', 'description']]

food_ds.head()

,fdc_id,description
0,2705383,"Milk, human"
1,2705384,"Milk, NFS"
2,2705385,"Milk, whole"
3,2705386,"Milk, reduced fat (2%)"
4,2705387,"Milk, low fat (1%)"


# Step 1 - Merge food_nutrient with nutrients


In [99]:
merged_foodNutrient_Nutrient = pd.merge(food_nutrient_ds, nutrient_ds, left_on='nutrient_id', right_on='nutrient_nbr', how='left')

# We verify that all the nutrients mentioned in food_nutrient_ds were found in nutrient_ds
assert len(merged_foodNutrient_Nutrient[merged_foodNutrient_Nutrient['nutrient_nbr'] == -1]) == 0, "There are unmatched nutrient_ids in food_nutrient_ds"

# The nutrient_nbr and nutrient_id columns are now redundant, we can drop one of them
merged_foodNutrient_Nutrient = merged_foodNutrient_Nutrient.drop(columns=['nutrient_nbr', 'nutrient_id'])
merged_foodNutrient_Nutrient.head()
# len(merged_foodNutrient_Nutrient)

,id,fdc_id,amount,Nutrient_Name,Nutrient_Unit
0,34136169,2705384,125.0,"Calcium, Ca",MG
1,34136178,2705384,57.0,Retinol,UG
2,34136185,2705384,0.0,Lycopene,UG
3,34136185,2705384,0.0,cis-Lycopene,UG
4,34136185,2705384,0.0,trans-Lycopene,UG


# Step 2 - Merge merged_nutrient_ds with food to get food descriptions

In [107]:
merged_foodNutrient_Nutrient_food = pd.merge(merged_foodNutrient_Nutrient, food_ds, left_on='fdc_id', right_on='fdc_id', how='left')

# Verify that all the fdc_ids mentioned in food_nutrient_ds were found in food_ds
assert len(merged_foodNutrient_Nutrient_food[merged_foodNutrient_Nutrient_food['description'].isnull()]) == 0, "There are unmatched fdc_ids in food_nutrient_ds"

# Drop id tables that are now redundant
merged_foodNutrient_Nutrient_food = merged_foodNutrient_Nutrient_food.drop(columns=['fdc_id', 'id'])

# merged_foodNutrient_Nutrient_food.head()

#Export the final dataset to a CSV file
merged_foodNutrient_Nutrient_food.to_csv(os.path.join(path, 'CATA_nutrients_per_aliment.csv'), index=False)



# Playground for exploring the data

In [109]:
def print_format(dataframes, index):
    print("Dataframe named:", csv_files[index])
    df = dataframes[index]
    return df

In [111]:
print_format(dataframes, 0).head()

Dataframe named: fndds_derivation.csv


,derivation code,derivation description
0,A,Analytical
1,AI,Analytical data; from the literature or gover...
2,AR,Analytical data; derived by linear regression
3,AS,Summed
4,BD,Based on same food; Drained solids from solids...


In [112]:
food = print_format(dataframes, 1).head()

stripped_food = food[['fdc_id', 'description']]

#Save as CSV
stripped_food.to_csv('./data/CATA_fdc_id_description_pairing.csv', index=False)


Dataframe named: food.csv


In [113]:
print_format(dataframes, 2).head()

Dataframe named: food_attribute.csv


,id,fdc_id,seq_num,food_attribute_type_id,name,value
0,3298314,2705383,NaN,999,WWEIA Category description,Human milk
1,3298313,2705383,NaN,999,WWEIA Category number,9602
2,3298316,2705384,NaN,999,WWEIA Category description,"Milk, reduced fat"
3,3298315,2705384,NaN,999,WWEIA Category number,1004
4,3318824,2705384,NaN,1002,NaN,Moisture change: 0%


In [114]:
food_atribute_ds = print_format(dataframes, 3)

food_atribute_ds['id'].describe()

Dataframe named: food_attribute_type.csv


count       5.000000
mean     1000.000000
std         1.581139
min       998.000000
25%       999.000000
50%      1000.000000
75%      1001.000000
max      1002.000000
Name: id, dtype: float64

In [115]:
print_format(dataframes, 5).head()

Dataframe named: food_portion.csv


,id,fdc_id,seq_num,amount,measure_unit_id,portion_description,modifier,gram_weight,data_points,footnote,min_year_acquired
0,290506,2705383,1,NaN,9999,1 cup,10205,246.0,NaN,NaN,NaN
1,290507,2705383,2,NaN,9999,Quantity not specified,90000,0.0,NaN,NaN,NaN
2,290508,2705383,3,NaN,9999,1 fl oz,30000,30.8,NaN,NaN,NaN
3,290513,2705384,5,NaN,9999,Guideline amount per cup of hot cereal,63546,61.0,NaN,NaN,NaN
4,290511,2705384,3,NaN,9999,1 individual school container,64294,244.0,NaN,NaN,NaN


In [116]:
print_format(dataframes, 6).head()

Dataframe named: food_update_log_entry.csv


,id,description,last_updated
0,2705383,"Milk, human",2024-10-31
1,2705384,"Milk, NFS",2024-10-31
2,2705385,"Milk, whole",2024-10-31
3,2705386,"Milk, reduced fat (2%)",2024-10-31
4,2705387,"Milk, low fat (1%)",2024-10-31


In [117]:
print_format(dataframes, 7).head()

Dataframe named: input_food.csv


,id,fdc_id,fdc_of_input_food,seq_num,amount,sr_code,sr_description,unit,portion_code,portion_description,gram_weight,retention_code
0,124268,2705384,NaN,1,40.0,1077,"Milk, whole, 3.25% milkfat, with added vitamin D",GM,0,NONE,40.0,0
1,124271,2705384,NaN,4,8.0,1085,"Milk, nonfat, fluid, with added vitamin A and ...",GM,0,NONE,8.0,0
2,124270,2705384,NaN,3,14.0,1082,"Milk, lowfat, fluid, 1% milkfat, with added vi...",GM,0,NONE,14.0,0
3,124269,2705384,NaN,2,38.0,1079,"Milk, reduced fat, fluid, 2% milkfat, with add...",GM,0,NONE,38.0,0
4,124272,2705385,NaN,1,100.0,1077,"Milk, whole, 3.25% milkfat, with added vitamin D",GM,0,NONE,100.0,0


In [118]:
print_format(dataframes, 8).head()

Dataframe named: measure_unit.csv


,id,name
0,1000,cup
1,1001,tablespoon
2,1002,teaspoon
3,1003,liter
4,1004,milliliter


In [119]:
print_format(dataframes, 10).head()

Dataframe named: survey_fndds_food.csv


,fdc_id,food_code,wweia_category_number,start_date,end_date
0,2705383,11000000,9602,2021-01-01,2023-12-31
1,2705384,11100000,1004,2021-01-01,2023-12-31
2,2705385,11111000,1002,2021-01-01,2023-12-31
3,2705386,11112110,1004,2021-01-01,2023-12-31
4,2705387,11112210,1006,2021-01-01,2023-12-31


In [120]:
print_format(dataframes, 11).head()

Dataframe named: wweia_food_category.csv


,wweia_food_category,wweia_food_category_description
0,1002,"Milk, whole"
1,1004,"Milk, reduced fat"
2,1006,"Milk, lowfat"
3,1008,"Milk, nonfat"
4,1202,"Flavored milk, whole"
